In [ ]:
#Importing the ImageDataGenerator class
import keras
from keras.preprocessing.image import ImageDataGenerator

#import model building libraries

#To import Sequential
from keras.models import Sequential
#To add Dense layers
from keras.layers import Dense
#To add Convolution layer
from keras.layers import Convolution2D
#To add  Maxpooling layer
from keras.layers import MaxPooling2D
#import flatten layer
from keras.layers import Flatten
import warnings
warnings.filterwarnings('ignore')

#import load_model from keras.model
from keras.models import load_model
#import image class from keras
from tensorflow.keras.preprocessing import image
#import numpy
import numpy as np
#import cv2
import cv2
from twilio.rest import Client
from playsound import playsound

#Define the parameters/arguments for ImageDataGenerator class
train_data = ImageDataGenerator(rescale= 1./255,shear_range= 0.2,rotation_range= 180,horizontal_flip = True,vertical_flip = True,zoom_range = 0.2)
test_data = ImageDataGenerator(rescale= 1./255)

#Applying ImageDataGenerator functionality to trainset
x_train = train_data.flow_from_directory(r"dataset/training_dataset",target_size = (128,128),
                                            class_mode = "binary",batch_size = 32)


#Applying ImageDataGenerator functionality to testset
x_test = test_data.flow_from_directory(r"dataset/test_dataset",target_size = (128,128),
                                            class_mode = "binary",batch_size = 32)

#Initializing the model
model=Sequential()

#add convolutional layer
model.add(Convolution2D(32,(3,3),activation = "relu",input_shape = (128,128,3)))
#add maxpooling layer
model.add(MaxPooling2D(pool_size = (2,2)))
#add flatten layer 
model.add(Flatten())
#add hidden layer
model.add(Dense(300,activation='relu'))
model.add(Dense(300,activation='relu'))
#add output layer

model.add(Dense(1, activation="sigmoid"))

#configure the learning process
model.compile(loss="binary_crossentropy", metrics=["accuracy"], optimizer='adam')

#Training the model
model.fit(x_train, epochs = 10, validation_data=x_test, steps_per_epoch=len(x_train), validation_steps=len(x_test))

#Save the model
model.save("forestfire.h5")

#load the saved model
model = load_model("forestfire.h5")

video=cv2.VideoCapture("sample_video.mp4")
while(1):
    success, frame=video.read()
    cv2.imwrite("image.jpg", frame)
    img=keras.utils.load_img("image.jpg", target_size=(128,128))
    x=keras.utils.img_to_array(img)
    x=np.expand_dims(x,axis=0)
    pred = model.predict(x)
    cv2.imshow("frame",frame)
    if cv2.waitKey(10000) & 0xFF== ord('q'):
        break
    if pred[0]==1:
        account_sid = 'ACbc91e3dd3e67b221d468f5b76b3151d3'
        auth_token = '401857e61dbe714cc6e7b25aa63fb937'
        client = Client(account_sid, auth_token)

        message = client.messages \
           .create(
                 body='Alert: Forest fire is detected!!',
                 from_='+16295295924',
                 to='+919360233080'
            )
            
        print(message.sid)
        print('Fire Detected')
        print('Alarm sound is playing')
        playsound(r'Siren.mp3')
        print ('SMS sent!')
        break
    else:
        print("No Danger") 
    
video.release()
cv2.destroyAllWindows()